# Spectral analyses using Sentinel-2 Data


## Getting started

First we load the required Python libraries and tools.

In [3]:
from pystac_client import Client
from dask.distributed import Client as DaskClient
from odc.stac import load, configure_s3_access
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import folium
import odc.geo.xr  # noqa: F401

## Study site configuration

Here we establish the STAC catalog we're using as well as a
spatial and temporal extent. This can be anywhere, but this location
near Kuching was chosen due to the training data having several
classes available.

In [5]:
# aoi = gpd.read_file("efate_mangroves.geojson")
# bbox = aoi.get_bounds()

In [6]:
# STAC Catalog URL
# catalog = "https://stac.staging.digitalearthpacific.org"
catalog = "https://earth-search.aws.element84.com/v1"
# Create a STAC Client
client = Client.open(catalog)

In [7]:
# min_lat = -21.235371
# min_lon = -175.160308
# max_lat = -21.225558
# max_lon = -175.152978

# bbox = [min_lon, min_lat, max_lon, max_lat]

In [8]:
bbox = aoi.total_bounds
print(bbox) 
# Output: array([min_x, min_y, max_x, max_y])

[168.30419763 -17.73166831 168.31361664 -17.71944101]


In [9]:
# Three months of data
datetime = "2022-06/2024-09"

In [10]:
# Create local dask cluster to improve data load time. Only run this once.
dask_client = DaskClient(n_workers=1, threads_per_worker=16, memory_limit='16GB')

# Configure S3 access. Cloud defaults is an optimisation, while requester pays is required for Landsat
configure_s3_access(cloud_defaults=True, requester_pays=True)

## Training data

Next up we gather training data. This could be any geospatial point dataset
with a column that is numeric, for the class.

If you'd like to explore the structure of this data, you can run `gdf.head()`
to see the first few rows. The `explore()` function with the `column` argument
will show the data on the map, and change the colour based on that column.

In [11]:
# # Get the training data
# # data_url = "https://raw.githubusercontent.com/nick-murray/coastTrain/main/data/coastTrain_v1_0.geojson"

# gdf = gpd.read_file('cordia_toloa_test.geojson', bbox=bbox)
# # gdf = gdf.fillna(0)
# gdf.explore(column="randomforest", legend=True)

## Find and load Sentinel-2 data

Here we search for Sentinel-2 scenes over our study area and use
Dask to lazy-load them. We're only loading the red, green, blue, nir and swir
bands, along with the scene classification (scl) band.

In [21]:
# Search for Sentinel-2 data
items = client.search(
    collections=["sentinel-2-c1-l2a"],
    bbox=bbox,
    datetime=datetime,
    query={"eo:cloud_cover": {"lt": 10}},
).item_collection()

print(f"Found {len(items)} items")

Found 46 items


In [22]:
# def mask_cloud(data, collection):
    
#     if collection == "sentinel-2-l2a":
#         scl_included_classes = [4, 5, 6, 7]
#         low_cloud = data.scl.isin(scl_included_classes)
#         data = data.where(low_cloud)

#     data = data.median(dim="time").compute()
#     data = data.expand_dims("time")
#     data = data.assign_coords(time=[pd.Timestamp("2024-01-01")])

#     return data

In [23]:
# Load the data into an xarray Dataset
data = load(
    items,
    measurements=["red", "green", "blue", "nir08", "swir16", "scl"],
    bbox=bbox,
    chunks={"x": 2048, "y": 2048},
    groupby="solar_day",
)

data

<xarray.Dataset> Size: 5MB
Dimensions:      (y: 138, x: 103, time: 29)
Coordinates:
  * y            (y) float64 1kB 8.038e+06 8.038e+06 ... 8.036e+06 8.036e+06
  * x            (x) float64 824B 8.505e+05 8.505e+05 ... 8.515e+05 8.515e+05
    spatial_ref  int32 4B 32758
  * time         (time) datetime64[ns] 232B 2022-12-06T23:01:14.924000 ... 20...
Data variables:
    red          (time, y, x) uint16 824kB dask.array<chunksize=(1, 138, 103), meta=np.ndarray>
    green        (time, y, x) uint16 824kB dask.array<chunksize=(1, 138, 103), meta=np.ndarray>
    blue         (time, y, x) uint16 824kB dask.array<chunksize=(1, 138, 103), meta=np.ndarray>
    nir08        (time, y, x) uint16 824kB dask.array<chunksize=(1, 138, 103), meta=np.ndarray>
    swir16       (time, y, x) uint16 824kB dask.array<chunksize=(1, 138, 103), meta=np.ndarray>
    scl          (time, y, x) uint8 412kB dask.array<chunksize=(1, 138, 103), meta=np.ndarray>

## Data preparation

Now that we have data, we need to clean it up, masking out clouds
and scaling values to between 0-1, which are the valid reflectance
values.

We add a couple of indices too, which will help the machine learning
algorithm.

Note that we still have a lazy-loaded array, and haven't transferred
any data over the network.

In [30]:
# Mask out clouds and scale values

# Apply Sentinel-2 cloud mask
# 1: defective, 3: shadow, 9: high confidence cloud, 10: thin cirrus
mask_flags = [1, 3, 9, 10]

cloud_mask = ~data.scl.isin(mask_flags)
masked = data.where(cloud_mask)

# Apply scaling and clip to valid data, from 0 to 1
scaled = (masked.where(masked != 0) * 0.0001).clip(0, 1)

# Add some indices
scaled["ndvi"] = (scaled.nir08 - scaled.red) / (scaled.nir08 + scaled.red)
scaled["ndwi"] = (scaled.green - scaled.nir08) / (scaled.green + scaled.nir08)
scaled["cmri"] = scaled.ndvi - scaled.ndwi

scaled


<xarray.Dataset> Size: 15MB
Dimensions:      (time: 29, y: 138, x: 103)
Coordinates:
  * y            (y) float64 1kB 8.038e+06 8.038e+06 ... 8.036e+06 8.036e+06
  * x            (x) float64 824B 8.505e+05 8.505e+05 ... 8.515e+05 8.515e+05
    spatial_ref  int32 4B 32758
  * time         (time) datetime64[ns] 232B 2022-12-06T23:01:14.924000 ... 20...
Data variables:
    red          (time, y, x) float32 2MB dask.array<chunksize=(1, 138, 103), meta=np.ndarray>
    green        (time, y, x) float32 2MB dask.array<chunksize=(1, 138, 103), meta=np.ndarray>
    blue         (time, y, x) float32 2MB dask.array<chunksize=(1, 138, 103), meta=np.ndarray>
    nir08        (time, y, x) float32 2MB dask.array<chunksize=(1, 138, 103), meta=np.ndarray>
    swir16       (time, y, x) float32 2MB dask.array<chunksize=(1, 138, 103), meta=np.ndarray>
    scl          (time, y, x) float32 2MB dask.array<chunksize=(1, 138, 103), meta=np.ndarray>
    ndvi         (time, y, x) float32 2MB dask.array<chunksize=(1, 138, 103), meta=np.ndarray>
    ndwi         (time, y, x) float32 2MB dask.array<chunksize=(1, 138, 103), meta=np.ndarray>
    cmri         (time, y, x) float32 2MB dask.array<chunksize=(1, 138, 103), meta=np.ndarray>

In [26]:
# Visualise one date, to make sure it looks good.
# This example shows empty areas where we've masked out nodata, but
# note that there are still a lot of clouds coming in!

scaled.isel(time=1).odc.explore(vmin=0, vmax=0.3)

## Create a cloud-free composite

The final data preparation step involves creating a temporal
median of the data bands. Here we use `compute()` to process
the data and bring it into memory.

We preview the data in the second cell below.

In [31]:
# Create a median composite, which should get rid of most of the remaining clouds
# Note that this will take a few minutes to complete

median = scaled.median("time").compute()

median

<xarray.Dataset> Size: 514kB
Dimensions:      (y: 138, x: 103)
Coordinates:
  * y            (y) float64 1kB 8.038e+06 8.038e+06 ... 8.036e+06 8.036e+06
  * x            (x) float64 824B 8.505e+05 8.505e+05 ... 8.515e+05 8.515e+05
    spatial_ref  int32 4B 32758
Data variables:
    red          (y, x) float32 57kB 0.2092 0.1583 0.1352 ... 0.152 0.1514
    green        (y, x) float32 57kB 0.238 0.1822 0.1653 ... 0.1612 0.1574
    blue         (y, x) float32 57kB 0.2158 0.1469 0.1319 ... 0.1404 0.1364
    nir08        (y, x) float32 57kB 0.4532 0.544 0.544 ... 0.4563 0.4134 0.4134
    swir16       (y, x) float32 57kB 0.3598 0.3466 0.3466 ... 0.3081 0.3081
    scl          (y, x) float32 57kB 0.0004 0.0004 0.0004 ... 0.0004 0.0004
    ndvi         (y, x) float32 57kB 0.3628 0.55 0.6064 ... 0.5016 0.467 0.4584
    ndwi         (y, x) float32 57kB -0.3268 -0.4993 -0.538 ... -0.4457 -0.4403
    cmri         (y, x) float32 57kB 0.6822 1.051 1.141 ... 0.947 0.9127 0.9043

In [36]:
median.ndvi.odc.explore(vmin=0, vmax=0.3)

In [35]:
median.ndwi.odc.explore(vmin=0, vmax=0.3)

In [34]:
median.cmri.odc.explore(vmin=0, vmax=0.3)

## Prepare training data array

This next step involves extracting observed values from the satellite data
and combining them with our point data, resulting in something like this:

`class, red, green, blue ...`

This structure is then fed into the machine learning classifier.

## Visualise our results

Here we're visualising the results along with the RGB image
and the original training data points. We're doing this using
a Python library called Folium.

In [93]:
# Put it all on a single interactive map
# center = [np.mean([min_lat[0], max_lat[0]]), np.mean([min_lat[1], max_lat[1]])]
# m = folium.Map(location=center, zoom_start=11)

center = [(min_lat + max_lat) / 2, (min_lon + max_lon) / 2]  # Assuming min_lon and max_lon are defined
m = folium.Map(location=center, zoom_start=11)



# RGB for the median
median.odc.to_rgba(vmin=0, vmax=0.3).odc.add_to(m, name="Median Composite")

# Categorical for the predicted classes and for the training data
predicted_da.odc.add_to(m, name="Predicted")
gdf.explore(m=m, column="randomforest", legend=True, name="Training Data")

# Layer control
folium.LayerControl().add_to(m)

m

In [94]:
predicted_da.odc.write_cog("classification_test.tif")
# predicted_da.plot.imshow()

PosixPath('classification_test.tif')

## Conclusions

Do the results make sense?

What are some of the limitations of the visualisation?

### Next steps and opportunities

The obvious next step is to fine tune the data. Perhaps download the points for this
region of interest as well as the RGB image and add and remove points until
there is a more representative training dataset.